## Multiple bots chatting with each other



Week2 Day 1 exercise (LLM Engineer course on Udemy by Ed Donner)

Reference: https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models

In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display


In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
grok_api_key = os.getenv("GROK_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

In [ ]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints

openai = OpenAI()

# For Gemini, DeepSeek and Groq, we can use the OpenAI python client
# Because Google and DeepSeek have endpoints compatible with OpenAI
# And OpenAI allows you to change the base_url

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
deepseek_url = "https://api.deepseek.com"
groq_url = "https://api.groq.com/openai/v1"
grok_url = "https://api.x.ai/v1"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
deepseek = OpenAI(api_key=deepseek_api_key, base_url=deepseek_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [ ]:
bot_names = ["Alex", "Blake", "Charlie"]


bot_models = {"Alex": "llama3.2", "Blake": "llama3.2", "Charlie": "llama3.2"}

bot_system_prompts = {
    "Alex": """
You are Alex, a snarky an confrontational chat bot, you challenge and confront anything said by others,
you are in conversation with Blake and Charlie, respond accordingly
""",
    "Blake": """
You are Blake, a jovial chat bot, crack jokes and dont take anything seriously,
you are in conversation with Alex and Charlie, respond accordingly
""",
    "Charlie": """
You are Charlie, a polite and formal chat bot, always try to get along with everyone,
you are in conversation with Alex and Blake, respond accordingly
""",
}

conversation = f"""
Alex: Hi
Blake: Hola, what a funny day!
Charlie: Hi there
"""


def get_user_prompt(bot_name):
    return f"""
    You are {bot_name} in conversation with other bots,
    Here is the conversation so far, each line starts with the name of the bot followed by :
    For example: if Alex says something, it will be like this:
    Alex: <message>
    If Blake says something, it will be like this:
    Blake: <message>
    If Charlie says something, it will be like this:
    Charlie: <message>
    When you respond, follow the format of the conversation so far, and respond accordingly
    {conversation}
    Now with this, respond accordingly
    """


def get_messages(bot_name):

    return [
        {"role": "system", "content": bot_system_prompts[bot_name]},
        {"role": "user", "content": get_user_prompt(bot_name)},
    ]


def get_response(bot_name):
    response = ollama.chat.completions.create(
        model=bot_models[bot_name], messages=get_messages(bot_name)
    )
    display(Markdown(response.choices[0].message.content))
    return response.choices[0].message.content + "\n"


for i in range(1):

    for bot_name in bot_names:
        conversation += get_response(bot_name)
        print(conversation)